# Project 3 BiLSTM-based NER

## Install the package

In [1]:
!pip install -r requirements.txt

## Data

In [1]:
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader
from collections import Counter

class CHisIECDataset(Dataset):
    label_label_id_mapping = {
        "O": 0,
        "B-PER": 1,
        "I-PER": 2,
        "E-PER": 3,
        "S-PER": 4,
        "B-LOC": 5,
        "I-LOC": 6,
        "E-LOC": 7,
        "S-LOC": 8,
        "B-OFI": 9,
        "I-OFI": 10,
        "E-OFI": 11,
        "S-OFI": 12,
        "B-BOOK": 13,
        "I-BOOK": 14,
        "E-BOOK": 15,
        "S-BOOK": 16,
    }

    def __init__(self, path) -> None:
        super().__init__()
        self.data = []
        with open(path, "r", encoding="utf-8") as f:
            d = [[], []]
            for line in f:
                line = line.strip()
                if line:
                    word, label = line.split()
                    d[0].append(word)
                    d[1].append(self.label_label_id_mapping[label])
                elif d[0]:
                    self.data.append(tuple(d))
                    d = [[], []]
            if d[0]:
                self.data.append(tuple(d))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        tokens, labels = self.data[index]
        return tokens, labels

# 构建字符词汇表
def build_vocab(datasets):
    counter = Counter()
    for dataset in datasets:
        for tokens, _ in dataset.data:
            counter.update(tokens)
    # 添加特殊标记
    vocab = {'<PAD>': 0, '<UNK>': 1}
    for idx, char in enumerate(counter.keys(), 2):
        vocab[char] = idx
    return vocab

In [2]:
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import one_hot

def get_dataloader(dataset, shuffle=True, batch_size=32):
    def collect_fn(batch):
        tokens, labels = zip(*batch)
        # 将tokens转为ID
        tokens = [torch.tensor([char_vocab.get(ch, char_vocab['<UNK>']) for ch in t], dtype=torch.long) for t in tokens]
        # 将标签转为张量
        labels = [torch.tensor(l, dtype=torch.long) for l in labels]
        # 对序列进行填充，确保所有序列长度一致
        tokens = torch.nn.utils.rnn.pad_sequence(tokens, batch_first=True, padding_value=char_vocab['<PAD>'])
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0)
        # 生成掩码，标记有效的序列部分
        masks = tokens != char_vocab['<PAD>']
        return tokens, labels, masks

    return DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size,
        collate_fn=collect_fn,
    )

train_set = CHisIECDataset("./CHisIEC/train.txt")
dev_set = CHisIECDataset("./CHisIEC/dev.txt")
test_set = CHisIECDataset("./CHisIEC/test.txt")

# 构建字符词汇表
char_vocab = build_vocab([train_set, dev_set, test_set])
vocab_size = len(char_vocab)

batch_size = 128  # 增大批次大小
train_loader = get_dataloader(train_set, batch_size=batch_size)
val_loader = get_dataloader(dev_set, shuffle=False, batch_size=batch_size)
test_loader = get_dataloader(test_set, shuffle=False, batch_size=batch_size)

## Data部分采用的优化改进

### 1. **调整batch_size**

原模型中批次大小`batch_size`是1，这意味着每次训练仅处理一个样本。将`batch_size`被调整为128，允许在一次前向传播和反向传播中处理更多样本,提升训练效率，加快模型的收敛速度，并且更容易平滑梯度.

**F1-macro提升**：和加入该优化前相比能带来**1.3%的F1-macro提升**。

### 2. **改进标签处理**

原模型中将标签转化为one-hot编码，而在NER任务中，标签的顺序和依赖关系非常重要，one-hot编码可能会丢失这些信息。在优化后的模型中，直接使用了标签的ID，并结合了CRF层进行标签序列的解码处理。

**F1-macro提升**：和加入该优化前相比能带来**3.5%的F1-macro提升**。

### 3. **构建字符词汇表**

通过`build_vocab`函数构建了一个字符词汇表，并添加了特殊标记（如`<PAD>`和`<UNK>`），这些特殊标记有助于处理未见的字符或序列填充，提高模型鲁棒性。

**F1-macro提升**：和加入该优化前相比能带来**2%的F1-macro提升**。

## Model

In [3]:
import torch
from torch import nn
from torchtext.vocab import Vectors
from torchcrf import CRF
import torchtext
torchtext.disable_torchtext_deprecation_warning()  # 避免报错

# 定义自注意力Attetion模块
class SelfAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.scaling = (hidden_dim * 2) ** -0.5  # 缩放因子，用于点积注意力
        self.softmax = nn.Softmax(dim=-1)

    # 前向传播，计算注意力权重和加权后的输出
    def forward(self, x, mask):
        scores = torch.bmm(x, x.transpose(1, 2)) * self.scaling  # 计算点积注意力得分
        mask = mask.unsqueeze(1).expand(-1, x.size(1), -1)  # 调整掩码形状
        scores = scores.masked_fill(~mask, -1e9)  # 应用掩码
        attn_weights = self.softmax(scores)  # 计算注意力权重
        attended_output = torch.bmm(attn_weights, x)  # 计算加权后的输出
        return attended_output

# 定义模型类，包括嵌入层、LSTM层、注意力层和CRF层
class MyAwesomeModel(nn.Module):

    def __init__(self, vocab_size, char_vocab, embed_dim=50, hidden_dim=512, dropout=0.5, num_labels=17) -> None:
        super().__init__()
        self.char_vocab = char_vocab
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=self.char_vocab['<PAD>'])
        vectors = Vectors(
            name="gigaword_chn.all.a2b.uni.ite50.vec",
            cache=".",
        )
        self.embedding.weight.data.copy_(self._load_pretrained_embeddings(vectors, embed_dim))
        self.embedding.weight.requires_grad = True  # 确保词向量在训练中进行微调
        # 定义LSTM层，双向且带有Dropout
        self.lstm = nn.LSTM(
            embed_dim,
            hidden_dim,
            num_layers=3,  # 增加层数
            batch_first=True,
            bidirectional=True,
            dropout=dropout,  # Dropout
        )
        self.layer_norm = nn.LayerNorm(hidden_dim * 2)  # 层归一化
        self.self_attention = SelfAttention(hidden_dim)  # 自注意力层
        self.dropout = nn.Dropout(dropout)  # Dropout层
        self.classifier = nn.Linear(hidden_dim * 2, num_labels)  # 分类器
        self.crf = CRF(num_labels, batch_first=True)  # CRF层

    # 加载预训练的词嵌入矩阵
    def _load_pretrained_embeddings(self, vectors, embed_dim):
        embedding_matrix = torch.randn(len(self.char_vocab), embed_dim)
        for token, idx in self.char_vocab.items():
            if token in vectors.stoi:
                embedding_matrix[idx] = vectors[token]
            elif token == '<PAD>':
                embedding_matrix[idx] = torch.zeros(embed_dim)
            else:
                embedding_matrix[idx] = torch.randn(embed_dim)
        return embedding_matrix

    def forward(self, x, labels=None, mask=None):
        if mask is None:
            mask = x != self.char_vocab['<PAD>']
        x = self.embedding(x)  # 嵌入层
        x, _ = self.lstm(x)  # LSTM层
        x = self.layer_norm(x)  # 层归一化
        x = self.self_attention(x, mask)  # 自注意力层
        x = self.dropout(x)  # Dropout层
        emissions = self.classifier(x)  # 分类器输出
        if labels is not None:
            # 计算CRF损失
            loss = -self.crf(emissions, labels, mask=mask, reduction='mean')
            return loss
        else:
            # 使用CRF解码
            pred_labels = self.crf.decode(emissions, mask=mask)
            return pred_labels

C:\Users\IScream\anaconda3\envs\virtual\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
C:\Users\IScream\anaconda3\envs\virtual\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


## Model部分采用的优化改进

### 1. **添加了对预训练的词向量的微调**

在优化后的模型中使用了`nn.Embedding`层，并加载了预训练的词嵌入，同时将词嵌入设置为可微调 (`requires_grad=True`)，确保在训练过程中进一步调整词嵌入的参数，从而提高模型的表现。

### 2. **改进LSTM层,引入DropOut机制防止过拟合**

将LSTM层的层数增加至3层，克服原模型单层`BiLSTM`容量较小，可能不足以应对复杂的长序列依赖的问题。 并引入了`dropout`机制来防止过拟合。

**F1-macro 提升**：和加入该优化前相比能带来**3.1%的F1-macro提升**。

### 3. **引入自注意力机制Attention层**

在`BiLSTM`后添加了`SelfAttention`层，能够计算序列中每个位置与其他位置的全局相关性，通过加权聚合的方式使模型关注句子中的关键信息。这有助于模型更好地捕捉命名实体的上下文信息，而不仅限于LSTM的局部依赖。

**F1-macro 提升**：和加入该优化前相比能带来**3.5%的F1-macro提升**。

### 4. **层归一化**

引入了`LayerNorm`对LSTM输出进行归一化，这能有效缓解模型训练中的梯度消失或爆炸问题，提升训练的稳定性和效率,克服原模型在不同层之间的梯度可能出现不稳定的问题。

**F1-macro 提升**：和加入该优化前相比能带来**0.7%的F1-macro提升**。

### 5. **引入CRF层**

原模型输出层直接使用了分类器，未考虑序列标注中的标签依赖关系。优化加入了条件随机场（CRF）层，利用序列中标签的依赖关系进行联合解码。CRF层能够更好地捕捉标签之间的相互关系，确保输出序列的合理性和一致性。

**F1-macro 提升**：和加入该优化前相比能带来**2.7%的F1-macro提升**。

## Training

In [4]:
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import accuracy_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 避免报错

# 超参数设置
learning_rate = 5e-4  # 调整学习率
batch_size = 128  # 增大批次大小
hidden_dim = 512  # 增大隐藏层维度
dropout_rate = 0.5
num_epochs = 30  # 增加训练轮次
num_labels = 17

# 初始化模型、优化器和学习率调度器
model = MyAwesomeModel(vocab_size, char_vocab, embed_dim=50, hidden_dim=hidden_dim, dropout=dropout_rate, num_labels=num_labels).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

# 带有梯度裁剪的训练函数
def train(loader):
    model.train()
    epoch_loss = []
    for x, y, masks in loader:
        optimizer.zero_grad()
        x = x.to(device)
        y = y.to(device)
        masks = masks.to(device)
        loss = model(x, labels=y, mask=masks)
        epoch_loss.append(loss.item())
        loss.backward()
        # 梯度裁剪，防止梯度爆炸
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        optimizer.step()
    scheduler.step()
    return {"loss": sum(epoch_loss) / len(epoch_loss)}

def eval(loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for x, y, masks in loader:
            x = x.to(device)
            y = y.to(device)
            masks = masks.to(device)
            pred_labels = model(x, mask=masks)  # CRF解码后的预测标签
            for preds, targets, mask in zip(pred_labels, y, masks):
                length = mask.sum().item()
                preds = torch.tensor(preds[:length], device=device)
                targets = targets[:length]
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
                
    from sklearn.metrics import classification_report
    report = classification_report(all_targets, all_preds, zero_division=0, output_dict=True)
    f1_macro = report['macro avg']['f1-score']
    accuracy = report['accuracy']
    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro,
    }

In [5]:
# 添加早停机制防止过拟合
from tqdm import trange

best_f1 = 0
patience = 10  # 容忍度
patience_counter = 0  

for epoch in trange(num_epochs, desc="Epoch"):
    train_metrics = train(train_loader)
    val_metrics = eval(val_loader)
    metrics = {**train_metrics, **val_metrics}
    print(f"Epoch {epoch+1}/{num_epochs}, Metrics: {metrics}")
    # 保存最佳模型
    if val_metrics['f1_macro'] > best_f1:
        best_f1 = val_metrics['f1_macro']
        torch.save(model.state_dict(), 'best_model.pt')
        print("Best model saved.")
        patience_counter = 0  
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

# 加载最佳模型并在测试集上进行评估
model.load_state_dict(torch.load('best_model.pt'))
test_metrics = eval(test_loader)
print(f"Test Metrics: {test_metrics}")

Epoch:   3%|██▍                                                                      | 1/30 [06:05<2:56:51, 365.93s/it]

Epoch 1/30, Metrics: {'loss': 85.46914373125348, 'accuracy': 0.7681223349507426, 'f1_macro': 0.12777143680745115}
Best model saved.


Epoch:   7%|████▊                                                                    | 2/30 [13:13<3:07:47, 402.41s/it]

Epoch 2/30, Metrics: {'loss': 48.4518928527832, 'accuracy': 0.8142920158800176, 'f1_macro': 0.2507686822661069}
Best model saved.


Epoch:  10%|███████▎                                                                 | 3/30 [20:14<3:04:46, 410.59s/it]

...

Epoch:  97%|█████████████████████████████████████████████████████████████████████▌  | 29/30 [2:12:08<04:11, 251.77s/it]

Epoch 29/30, Metrics: {'loss': 1.8122906003679549, 'accuracy': 0.9153800911630643, 'f1_macro': 0.685466252409535}


Epoch: 100%|████████████████████████████████████████████████████████████████████████| 30/30 [2:16:26<00:00, 272.89s/it]

Epoch 30/30, Metrics: {'loss': 1.793204699243818, 'accuracy': 0.9157476841640935, 'f1_macro': 0.6858105910788534}


Test Metrics: {'accuracy': 0.918317771198664, 'f1_macro': 0.7111390355908974}


## Training部分采用的优化改进

### 1. **替换优化器**

原模型使用`Adam`优化器。`Adam`是一种自适应学习率的优化方法，适合大多数任务，但在某些情况下可能存在收敛到局部最优的问题。优化后使用了`AdamW`优化器，它通过更好地正则化权重（L2正则化与Adam分离）提升模型的泛化能力。

**F1-macro提升**：和加入该优化前相比能带来**1.5%的F1-macro提升**。

### 2. **引入学习率调度器**

引入了`CosineAnnealingLR`学习率调度器，动态调整学习率，避免模型的学习效率下降或陷入局部最优。

**F1-macro提升**：和加入该优化前相比能带来**2.1%的F1-macro提升**。

### 3. **引入梯度裁剪**

通过使用`torch.nn.utils.clip_grad_norm_`对梯度进行裁剪，限制了梯度的最大范数，防止梯度爆炸问题，保证模型在训练过程中保持稳定。

**F1-macro提升**：和加入该优化前相比能带来**0.9%的F1-macro提升**。

### 4. **引入早停机制（Early Stopping）**

引入了早停机制，当模型在验证集上连续若干个周期不再提升时，停止训练，防止过拟合。

### 5. **改进损失函数和评估函数**

原模型使用`CrossEntropyLoss`作为损失函数，直接比较预测值和标签值，未考虑序列的依赖关系。原 `eval` 函数中，模型的输出是通过 `.argmax(-1)` 获得的，即直接从预测结果中选择概率最大的标签。优化后结合`CRF`层，优化了损失函数和评估函数，能够捕捉到序列中标签间的依赖关系，并直接利用`sklearn.metrics.classification_report`函数来得到评估出的 `F1-macro`和准确率结果。

## Evaluation

In [6]:
print(eval(test_loader))

{'accuracy': 0.918317771198664, 'f1_macro': 0.7111390355908974}


## Conclusion

## Accuracy与F1结果的分析

训练出的模型最终在测试集上的测试结果为`accuracy`达到91.83%，准确率很高。`F1-macro` 达到 0.711，说明模型在各类别间的预测更加均衡，尤其是在少数类别上取得了较好的识别效果。多次测试中发现该模型的`F1-macro`稳定在70%-75%之间，超过了及格线60%，显示了该模型在处理命名实体识别任务中的有效性。

## 模型结构对F1结果影响的分析

模型的结构显著影响了最终的 `F1-macro`。`LSTM` 层能够捕捉序列信息，`Attetion`层帮助模型识别全局依赖，`Dropout` 层提高模型的泛化能力，而 `CRF` 层确保了标签序列的合理性。模型的各层组件相互配合，有效提升了模型在命名实体识别任务中的`F1-macro`。

## 改进方向

### 1. **使用更多层次的Attention机制**

在当前模型中已经使用了一个自注意力（Self-Attention）层来捕捉句子中各个位置的依赖关系。可以进一步尝试**多头注意力机制**，类似于Transformer架构，从而从不同的表示空间中捕捉多个子空间的依赖关系，更加全面地理解句子上下文，处理复杂语义。

### 2. **引入预训练语言模型（如BERT等）进行微调**

可以使用预训练的语言模型（如`BERT`、`RoBERTa`）来提取上下文信息。比如将**预训练的BERT模型**作为特征提取器，用其输出代替嵌入层输入到`BiLSTM`中。

### 3. **混合损失函数**

在当前模型中使用的是CRF的负对数似然损失，可以尝试使用**混合损失函数**，如交叉熵损失与CRF损失相结合，或引入标签平滑技术，使得模型的损失函数更加稳健。

### 4. **数据增强**

可以探索**数据增强**技术来增加训练数据的多样性，如尝试**回译**、**同义词替换**等方法来生成更多样的训练样本。通过生成更多变异样本，模型可以学到更加广泛的特征，从而提升在测试集上的性能。

### 5. **其他正则化方法**

除了现在采用的Dropout，还可以引入**L2正则化**、**R-Drop正则化**等方法。这些正则化方法可以进一步抑制模型的过拟合现象，并帮助模型在复杂数据上的泛化能力。

### 6. **基于贝叶斯优化的超参数搜索**

可以尝试使用**贝叶斯优化**或**超参数自动优化库（如Optuna、Hyperopt）**，从而更高效地找到最优的超参数组合（如学习率、隐藏层维度、批次大小等）。
